In [12]:
import torch
from PIL import Image
from torchvision import transforms
from torchvision.models import efficientnet_v2_s

In [2]:
model = efficientnet_v2_s()

In [13]:
img = Image.open("C:/Users/josem/Documents/schoolWork/MQP/algonauts2023_transformers#2Leader/algonauts_2023_challenge_data/subj01/training_split/training_images/train-0013_nsd-00140.png")

In [15]:
tsfms = transforms.Compose([
    transforms.Resize(384),
    transforms.ToTensor()
])

In [10]:
features = model.features[:-1]
finalConvLayer = list(model.features[-1].children())[0]

In [23]:
extracted = features(tsfms(img)[None, :, :, :])



In [26]:
torch.flatten(finalConvLayer(extracted), 1).shape

torch.Size([1, 184320])

In [5]:
class EfficientNetExtractor(torch.nn.Module):
    def __init__(self):
        super(EfficientNetExtractor, self).__init__()
        self.efficientNet = efficientnet_v2_s()
        self.features = self.efficientNet.features[:-1]
        self.finalConvLayer = list(self.efficientNet.features[-1].children())[0]

    def forward(self, img):
        features = self.features(img)
        features = self.features(features)
        
    

Conv2dNormActivation(
  (0): Conv2d(256, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(1280, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): SiLU(inplace=True)
)